# Deploy pre-trained model to Amazon SageMaker
_**Hosting a pre-trained scikit-learn Model in Amazon SageMaker scikit-learn Container**_

---

---

## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-trained scikit-learn model with the Amazon SageMaker scikit-learn container to quickly create a hosted endpoint for that model.
We use the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html. The California Housing dataset was originally published in:

> Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data.
* The S3 bucket that you want to use for training and model data.

In [ ]:
!pip install -U sagemaker

In [ ]:
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sagemaker.sklearn import SKLearnModel, SKLearn


region = boto3.Session().region_name
role = get_execution_role()

In [ ]:
bucket = "jenfi-sagemaker-models"
prefix = "models/vn_two_mob_fifteen_plus"

print(f"bucket: {bucket}")

### Prepare the model file

`model_handler.py` will take a fixed name `model.pickle` when decompressed. We need to store the `.pickle` with following path:

```models/model_folder_name/model.pickle```

In [ ]:
model_file_path = "./models/vn_two_mob_fifteen_plus/model.pickle"

### Compressed the model file to a GZIP tar archive 

Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file needs to be tar-zipped. 

In [ ]:
model_tar_path = "./models/vn_two_mob_fifteen_plus/model.tar.gz"

In [ ]:
!tar czvf $model_tar_path $model_file_path

## Upload the pre-trained model `model.tar.gz` file to S3

In [ ]:
fObj = open(model_file_path, "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

## Set up hosting for the model

This involves creating a SageMaker model from the model file previously uploaded to S3.

In [ ]:
model_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {model_data}")

### Deploy with Python SDK

Here we showcase the process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [ ]:
model_name = 'VnTwoMobFifteenPlus'

# endpoint_name will be used to invoke model
endpoint_name = 'VnTwoMobFifteenPlus'

In [ ]:
vpc_config = {
    "SecurityGroupIds": [
        "sg-02610d5a03c041da3",
        "sg-03fee2a73bcea76b6"
    ],
    "Subnets": [
        "subnet-0dd112352f47d8897",
        "subnet-04b844519774a51a1",
        "subnet-043ccd31143339dc5"
    ],
}

vpc_config

In [ ]:
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="1.2-1",
    py_version="py3",
    entry_point="model_handler.py",
    name=model_name,
    vpc_config=vpc_config,
)

### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 5-10 minutes to complete.

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    capture_options=['REQUEST', 'RESPONSE'],
    destination_s3_uri='s3://jenfi-sagemaker-data-capture',
    sampling_percentage=100
)


In [ ]:
%%time

predictor = model.deploy(
    instance_type="ml.t2.medium",
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    data_capture_config=data_capture_config,
    tags=[{'Key': 'version', 'Value': '3.0'}],
)

## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

### Invoke with the Python SDK

Let's generate the prediction for a single data point. We'll pick one from the test data generated earlier.

In [ ]:
row_data = {
    'bal_month_level_min_lag_month_1_week_3_overall_level_0': 1,
    'db_amt_month_level_median_lag_month_2_week_1_overall_level_0': 1,
    'burn_amt_month_level_max_lag_month_2_week_2_overall_level_0': 0,
    'outflow_inflow_ratio_cnt_day_level_lag_27_overall_level_0': 0,
    'burn_amt_day_level_lag_43_corporate_level_0': 0
}

feat_dataframe = pd.DataFrame([row_data])

feat_dataframe

In [ ]:
from statsmodels.api import add_constant
input_predict = add_constant(feat_dataframe, has_constant='add')

In [ ]:
input_predict_json = input_predict.to_json(orient='records', lines=True).encode("utf-8")

In [ ]:
input_predict_json

In [ ]:
runtime = boto3.client("sagemaker-runtime")

In [ ]:
endpoint_name

In [ ]:
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    Body=input_predict_json,
    ContentType="application/json",
)

In [ ]:
response["Body"].read()